In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from spectral_connectivity import Multitaper
from spectral_connectivity import Connectivity
from core_functions.simulated_data import simdata_settings, simulate
from core_functions.core import em_B, get_theoretical_psd_COM


In [2]:
# Simulate Model
T = 30000
blnSimS = False

fs, k, n, M, osc_freqs, rhos, var_state_nois, var_obs_noi, A, Q, R, Btrue, Z, X0, S0 = simdata_settings()

ta = np.arange(T)/fs
x_dim = k*2

if blnSimS:
    s=None
else:     
    s = np.zeros(T,dtype=int)
    s[ta>80]=1
    s[ta>200]=2

y, s, x = simulate(A, Btrue, Q, R, Z, X0, S0, T, s=s)


In [3]:
# Pretend we don't know B and fit it
tol = 1e-6 # Tolerance
max_iter = 10 # Max iterations

B0 = np.random.uniform(0,0.1,size=[n,x_dim,M])

em_B(y, tol, max_iter, A, B0, Q, R, Z, X0)

LinAlgError: Singular matrix